In [103]:
import pandas as pd
import numpy as np
from plotnine import *

# Pipeline:

- prepare sample table
- run trimming to remove adapters and primers
- run dada2 to generate ASVs, remove chimeras, and abundance table
- run nextclade to call SNVs and indels


TO-DO:

Turn pipeline into an executable that takes sample table as input

# Set up sample directory
1. download reads into data/raw
2. make read_list.txt (ls \*R1\*.fastq.gz > reads_list.txt)
3. make subdirectories within data/raw for each amplicon
4. make trimmed_reads directories for each amplicon (e.g. data/raw/trimmed_reads)

# Make sample table (can do this in a spreadsheet or here)

**sample table must have columns:**

- read1 # basename of read file 1 (e.g. "NTD_I_I_INF_101321_1_S58_L001_R2_001.fastq.gz")
- read2 # created off of read1
- amplicon # extracted from read1 (e.g. "NTD")
- sample_id # extracted from read1, following lab convention (e.g. "I_I_INF_101321_1")
- merge_id # same as sample_id, or a replicate sample with a Cq value in qPCR table (if the sample sent for sequencing was not analyzed via qPCR)
- treatment # any upstream non-standard treatment performed on the sample or RNA (e.g. "rRd" for RNA depletion)
- datadir # location of read1 and read2 files

In [229]:
# 10/4/21
# # samples given to Justin were run with RBD primers only and were processed with rRNA depletion ('rRd') or without ('STD')
# # samples run in-house were labeled "RBD"

datadir = '/Users/rosekantor/data/wbe_scv/qb3_sgene_100421'
reads_df = pd.read_csv(f'{datadir}/raw/reads_list.txt', names=['read1'])

reads_df = reads_df[~reads_df.read1.str.contains('Undetermined')]
reads_df['read2'] = reads_df.read1.str.replace(r'_R1_', '_R2_')
reads_df[['amplicon','sample_id']] = reads_df.read1.str.extract(r'(\w+)_(.+_.+_.+_\d+_\d+)_S\d+_L001_R._001\.fastq\.gz')
reads_df['merge_id'] = reads_df['sample_id'].copy()
reads_df['treatment'] = 'inhouse'
reads_df.loc[reads_df.amplicon == 'rRd', 'treatment'] = 'rRd'
reads_df.loc[reads_df.treatment == 'rRd', 'amplicon'] = 'RBD' # rename amplicon, all ribodepleted samples were RBD this round
reads_df.loc[reads_df.amplicon == 'STD', 'treatment'] = 'std'
reads_df.loc[reads_df.amplicon == 'STD', 'amplicon'] = 'RBD' # the non-ribodepleted amplicons from FGL were RBD but got named "STD" for standard treatment
reads_df['datadir'] = datadir

# In this run, the same samples were run with and without ribodepletion. Give these unique names:
reads_df.loc[reads_df.treatment.isin(['std', 'rRd']), 'sample_id'] = reads_df.treatment + "_" + reads_df.sample_id

reads_df.to_csv(f'{datadir}/sample_table.csv', index=False)

In [215]:
# 11/5/21
# samples all processed with rRNA depletion at FGL/GSL

datadir = '/Users/rosekantor/data/wbe_scv/qb3_sgene_110521'
reads_df = pd.read_csv(f'{datadir}/raw/reads_list.txt', names=['read1'])

reads_df = reads_df[~reads_df.read1.str.contains('Undetermined')]
reads_df['read2'] = reads_df.read1.str.replace(r'_R1_', '_R2_')
reads_df['read_name'] = reads_df.read1.str.replace('control', 'control_control') # control got misnamed, needs to say control_control to match pattern
reads_df['read_name'] = reads_df.read_name.str.replace('Twist_2_23', 'control_Twist_2_23_101521') # to match pattern
reads_df[['amplicon','sample_id']] = reads_df.read_name.str.extract(r'(\w+)_(.+_.+_.+_\d+_\d+)_S\d+_L001_R._001\.fastq\.gz')
reads_df['merge_id'] = reads_df['sample_id'].copy()
reads_df['treatment'] = 'rRd' # all were ribodepleted
reads_df['datadir'] = datadir

reads_df.to_csv(f'{datadir}/sample_table.csv', index=False)

# Generate commands

In [232]:
reads_df = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_sgene_110521/sample_table.csv')
# reads_df = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_sgene_100421/sample_table.csv')

In [217]:
# trim adapters and primers with cutadapt, specific to the primers used

# dict of fwd and rev trimming sequences
trim_seqs = {'RBD':['GTGATGAAGTCAGACAAATCGC...CAGACACTTGAGATTCTTGACAT', 
                    'ATGTCAAGAATCTCAAGTGTCTG...GCGATTTGTCTGACTTCATCAC'],
             'NTD':['GCGATTTGTCTGACTTCATCAC...TTTCGGCTTTAGAACCATTGG',
                    'CCAATGGTTCTAAAGCCGAAA...AAGAACAAGTCCTGAGTTGAATG'],
             'S1S2':['CAGGCACAGGTGTTCTTACT...CTACCAGTGTCTATGACCAAGAC',
                     'GTCTTGGTCATAGACACTGGTAG...AGTAAGAACACCTGTGCCTG']}

trim_cmd = []
for r in reads_df.itertuples():
    raw_path = f'{r.datadir}/raw'
    amplicon = r.amplicon
    trimmedF = f'{raw_path}/{amplicon.lower()}/trimmed_reads/{r.sample_id}.1.fastq.gz'
    trimmedR = f'{raw_path}/{amplicon.lower()}/trimmed_reads/{r.sample_id}.2.fastq.gz'
    
    cutadapt_cmd = f'cutadapt -a {trim_seqs[amplicon][0]} -A {trim_seqs[amplicon][1]} --discard-untrimmed -o {trimmedF} -p {trimmedR} {raw_path}/{r.read1} {raw_path}/{r.read2}'
    
    trim_cmd.append(cutadapt_cmd)

reads_df['trim_cmd'] = trim_cmd

# save commands
datadir = reads_df.datadir.values[0]
reads_df.trim_cmd.to_csv(f'{datadir}/read_trimming.sh', index=False, header=False)

In [233]:
# write commands for dada2 pipeline and nextclade

amplicons = ['NTD', 'RBD', 'S1S2']
datadir = reads_df.datadir.values[0]
dada2_pipeline = '/Users/rosekantor/work/wbe_sequence_analysis/dada2_pipeline.R'
nextclade = '/Users/rosekantor/work/wbe_sequence_analysis/nextclade'
nextclade_ref = '/Users/rosekantor/work/wbe_sequence_analysis/nextclade_data/sars-cov-2'
dada2_out = f'{datadir}/results/dada2_out'
nextclade_out = f'{datadir}/results/nextclade_out'

# NOTE: trimmed reads from each amplicon need to be in separate folders for dada2 to run on one amplicon at a time

with open(f'{datadir}/analysis_cmds.sh', 'w') as f:
    # option ot update nextclade and ref dataset
    # f.write('cd /Users/rosekantor/work/wbe_sequence_analysis\n')
    # f.write('curl -fsSL "https://github.com/nextstrain/nextclade/releases/latest/download/nextclade-MacOS-arm64" -o "nextclade" && chmod +x nextclade\n')
    # f.write('/Users/rosekantor/work/wbe_sequence_analysis/nextclade dataset get --name "sars-cov-2" --output-dir "nextclade_data/sars-cov-2"\n')

    f.write(f'mkdir {datadir}/results\n')
    f.write(f'mkdir {dada2_out}\n')
    f.write(f'mkdir {nextclade_out}\n')
    for amplicon in amplicons: 
        trimmed_path = f'{datadir}/raw/{amplicon.lower()}/trimmed_reads/'
        dada2_cmd = f'/Users/rosekantor/work/wbe_sequence_analysis/dada2_pipeline.R {amplicon} {trimmed_path} {dada2_out}'
        nextclade_cmd = f'{nextclade} --in-order --input-fasta {dada2_out}/{amplicon}_dada2_out.fasta --input-dataset {nextclade_ref} --output-csv {nextclade_out}/{amplicon}_nextclade.csv --output-dir {nextclade_out}'
        f.write(dada2_cmd + '\n')
        f.write(nextclade_cmd + '\n')
f.close()